# Implementing Dirac...
This is a SageMath script.   

In [19]:
import numpy as np

## Looking at Pauli matrices

In [20]:
one = matrix([[1,0],[0,1]])
zero = matrix([[0,0], [0,0]])
p1 = matrix([[0,1],[1,0]])
p2 = matrix([[0,-1j],[1j,0]])
p3 = matrix([[1,0],[0,-1]])

In [21]:
def pauli(i):
    d = {1: p1, 2: p2, 3: p3}
    return d[i]
show(pauli(2))

[  0.000000000000000 -1.00000000000000*I]
[ 1.00000000000000*I   0.000000000000000]

## Compose the four gamma matrices

In [22]:
def compose(a,b,c,d):
    # a b
    # c d
    return matrix(
        np.vstack([
            np.hstack([a,b]),
            np.hstack([c,d])
    ]))
show(compose(one, p1, p2, p3))

[   1.0    0.0    0.0    1.0]
[   0.0    1.0    1.0    0.0]
[   0.0 -1.0*I    1.0    0.0]
[ 1.0*I    0.0    0.0   -1.0]

In [23]:
def gamma(i):
    # The four gamma matrices
    if i == 0:
        return compose(one, zero, zero, -one)
    else:
        return compose(zero, pauli(i), -pauli(i), zero)
show(gamma(0), gamma(1), gamma(2), gamma(3))

[ 1  0  0  0]
[ 0  1  0  0]
[ 0  0 -1  0]
[ 0  0  0 -1] [ 0  0  0  1]
[ 0  0  1  0]
[ 0 -1  0  0]
[-1  0  0  0] [   0.0    0.0    0.0 -1.0*I]
[   0.0    0.0  1.0*I    0.0]
[   0.0  1.0*I    0.0    0.0]
[-1.0*I    0.0    0.0    0.0] [ 0  0  1  0]
[ 0  0  0 -1]
[-1  0  0  0]
[ 0  1  0  0]

## Check the anticommuator properties of the gamma matrices

In [24]:
def anticom(a,b):
    return a*b + b*a

In [25]:
rows = [[anticom(gamma(i), gamma(j)) for i in range(4)] for j in range(4)]
show(table(rows))

,,,
,,,
,,,
,,,


In [26]:
# Check for a random 4-vector
r = vector([4, 7, 0, 2])
r

(4, 7, 0, 2)

In [27]:
z = sum(r[i] * gamma(i) for i in range(4))
z*z

[-37.0   0.0   0.0   0.0]
[  0.0 -37.0   0.0   0.0]
[  0.0   0.0 -37.0   0.0]
[  0.0   0.0   0.0 -37.0]

## Defining a 4-spinor wave function
This is intended to be a solution to the Dirac equation.  It would be nice to model this as an explicit function of  (t, r).  However, SageMath doesn't allow functions returning a vector, so we have to model as a simple variabe, dependent on the variable t and r.  Not so clear....

In [28]:
t = var('t')
m = var('m')
x,y,z = var("x,y,z")
r = vector([x,y,z])

def generate_dirac_psi(p, E, m):
    # Generate a 4-spinor state for a matter particle of momentum p, total energy E, mass m, spin up.
    # For this to satisfy the Dirac Equation, we requre E^2 = p^2 + m^2
    assert (E^2 == p.dot_product(p) + m^2)
    
    U = sqrt((E+m) / (2*E)) * vector ([
            1, 
            0, 
            p[2]/(E+m), 
            (p[0] + i * p[1])/(E+m)
        ])
    return U * exp(i*(p.dot_product(r) - E*t))

In [29]:
# Generate a specific wave function to test
M=4  # Mass of our particle
dirac_psi = generate_dirac_psi(p=vector([3,0,0]), E=5, m=M)
show(dirac_psi)

(3*sqrt(1/10)*e^(-5*I*t + 3*I*x), 0, 0, sqrt(1/10)*e^(-5*I*t + 3*I*x))

## The Dirac differential operator
This implements the key operator $i \hbar \gamma^\mu \partial_\mu$. (But we use $\hbar=1$)

Equivalent to $i \hbar \sum_{i=0}^{i=3} \gamma(i) \frac{\partial}{\partial X^i}$ where

$\gamma(i)$ is one of the four gamma matrices (composed from Pauli matrices) and 

$X^i$ is the $i$th spacetime coordinate : $X^0 = t, (X^1, X^2, X^3) = (x, y, z) = r$.

In [30]:
# Apply the Dirac differentrial operator to the given wave function
def dirac_operator(psi):
    X = vector([t,r[0], r[1], r[2]])
    result = i * sum(gamma(q) * diff(psi, X[q]) for q in range(4))
    return result

## The Dirac equation

$(i \hbar \gamma^\mu \partial_\mu - m) \Psi = 0$, or

$(i \hbar \gamma^\mu \partial_\mu \Psi - m \Psi = 0$

Below we evaluate the LHS of this equation, using our carefully constructed wave function.

In [36]:
dirac_lhs = dirac_operator(dirac_psi) - m * dirac_psi

In [37]:
show(dirac_lhs)
# This is a function of t,x,y,z and m

(-3*sqrt(1/10)*m*e^(-5*I*t + 3*I*x) + 12*sqrt(1/10)*e^(-5*I*t + 3*I*x), 0, 0, -sqrt(1/10)*m*e^(-5*I*t + 3*I*x) + 4*sqrt(1/10)*e^(-5*I*t + 3*I*x))

In [38]:
# But if we substitute the value of m (= constant M) which went into the constructed wave function...
show(dirac_lhs.substitute(m=M))
# ... we should get zero.

(0, 0, 0, 0)

In [39]:
if dirac_lhs.substitute(m=M) == vector([0,0,0,0]):
    print("Looks like Dirac was right after all!")
else:
    print("Dirac was wrong (can't possibly be me!)")

Looks like Dirac was right after all!


In [40]:
# Show the first [0] component of the spinor, in x,y, varying t
@interact
def _(T=slider(np.linspace(0, 4, 21)), S=[0,1,2,3]):
    pt = contour_plot(real_part(dirac_psi[S].substitute(t=T, z=0)), (x, -5, 5), (y,-5, 5), axes_labels=['x', 'y'])
    show(pt)

Interactive function <function _ at 0x7f2279ece310> with 2 widgets
  T: SelectionSlider(value=0.0, options=(0.0, 0.2, 0.4, 0.6000000000000001, 0.8, 1.0, 1.2000000000000002, 1.4000000000000001, 1.6, 1.8, 2.0, 2.2, 2.4000000000000004, 2.6, 2.8000000000000003, 3.0, 3.2, 3.4000000000000004, 3.6, 3.8000000000000003, 4.0), description='T')
  S: Dropdown(value=0, options=(0, 1, 2, 3), description='S')